In [ ]:
import numpy as np

UNIVERSE_FILE="universe.csv"
MODEL_NAME_PREFIX = "universal_60_10_v1"

for num in np.arange(35, 36, 15):
    
    MODEL_NAME = f"universal_60_10_v1_{num}_features"
    DATA_SOURCE_MODEL = f"{MODEL_NAME}"
    
    !python -m _tools.feature_selector \
        --model_name "{MODEL_NAME}" \
        --top_n {num} \
        --corr_threshold 1.0 \
        --repeats 1 \
        --n_splits 5 \
        --use_gpu

    !python -m _tools.assemble_final_data \
        --model_name "{MODEL_NAME}" \
        --source_dir_name 'features_processed' \
        --workers 7

    !python -m _tools.fit_scaler \
        --model_name "{MODEL_NAME}"

    !python -m _tools.convert_to_tfrecord \
        --model_name "{MODEL_NAME}" \
        --lookback 60 \
        --workers 6

    !python -m _tools.train_universal_lstm \
        --model_name "{MODEL_NAME}" \
        --data_model_name "{DATA_SOURCE_MODEL}" \
        --batch_size 8192 \
        --lr 1e-2 \
        --l2_reg 1e-5 \
        --leaky_relu_negative_slope 0.0 \
        --iterations 1

In [10]:
MODEL_NAME = f"universal_60_10_v1_20_features"

In [31]:
!python -m _tools.assemble_final_data \
    --model_name "{MODEL_NAME}" \
    --source_dir_name 'features_processed'

--- ⚙️ Запуск сборки финального датасета ---
Источник файлов с признаками: /home/restorator/trader_test/2_results/features_processed
Загружено 20 отобранных признаков.
Загрузка и объединение 244 файлов в 4 потоков...
100%|█████████████████████████████████████████| 244/244 [00:02<00:00, 94.59it/s]
✅ Сборка завершена. В итоговой таблице 23 колонок (20 признаков).
Точка разделения train/test: 2024-09-19
Иммутация NaN завершена. NaN в train: 0 → 0; NaN в test: 0 → 0

✅ Обучающий набор сохранен в: /home/restorator/trader_test/2_results/final_for_model/universal_60_10_v1_20_features/train_final.csv
✅ Тестовый набор сохранен в: /home/restorator/trader_test/2_results/final_for_model/universal_60_10_v1_20_features/test_final.csv


In [53]:
!python -m _tools.fit_scaler \
    --model_name "{MODEL_NAME}"

————————————————————————————————————————————————————————————
🧰 Обучение скейлера (Imputer + RobustScaler)
Модель: universal_60_10_v1_20_features
Загрузка тренировочных данных из: /home/restorator/trader_test/2_results/final_for_model/universal_60_10_v1_20_features/train_final.csv
————————————————————————————————————————————————————————————
Признаков: 20, обучающих сэмплов: 390902
Обучение Pipeline (Imputer + RobustScaler)...
✅ Pipeline успешно обучен.
✅ Абсолютно 'безопасный' Pipeline сохранен в: /home/restorator/trader_test/2_results/scalers/universal_60_10_v1_20_features_scaler.pkl
🗂  Порядок признаков сохранен в: /home/restorator/trader_test/2_results/scalers/universal_60_10_v1_20_features_features.json
Готово.


In [54]:
!python -m _tools.apply_scaler \
    --model_name "{MODEL_NAME}"

---  Applying Scaler for model: universal_60_10_v1_20_features ---
Loading artifacts and data...
Applying scaler transformation...
Saving scaled data...
✅ Scaled data saved to: /home/restorator/trader_test/2_results/features_scaled/universal_60_10_v1_20_features


In [55]:
!python -m _tools.diagnose_scaler_application \
    --model_name "{MODEL_NAME}"

--- 🩺 Диагностика применения скейлера ---
Модель: universal_60_10_v1_20_features
Загружаю скейлер из: /home/restorator/trader_test/2_results/scalers/universal_60_10_v1_20_features_scaler.pkl
Загружаю список признаков из: /home/restorator/trader_test/2_results/scalers/universal_60_10_v1_20_features_features.json
✅ Артефакты успешно загружены. Признаков для масштабирования: 20
Загружаю данные из: /home/restorator/trader_test/2_results/final_for_model/universal_60_10_v1_20_features/train_final.csv
✅ Данные загружены. Форма: (390902, 23)
\n--- 🚀 Начинаю применение scaler.transform() к данным... ---
✅ scaler.transform() успешно выполнен!
\n--- 📊 Анализ результата масштабирования ---
                           mean       std       min         max
ICS_26                 0.244591  1.363050 -0.007985   26.212420
imoex_close            0.029673  0.303975 -0.591923    0.773326
usdrub_close           0.052172  0.332855 -0.943878    1.326955
sp500_close            0.026349  0.317700 -0.551716    0.

In [56]:
!python -m _tools.convert_to_tfrecord \
    --model_name "{MODEL_NAME}" \
    --lookback 60

I0000 00:00:1759477222.971656  878438 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1759477224.448253  878438 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
--- Converting scaled data to TFRecord for model: universal_60_10_v1_20_features ---
Loading scaled data...
Writing test.tfrecord: 100%|██████████████████| 243/243 [00:19<00:00, 12.73it/s]
Calculating class weights and saving metadata...
✅ TFRecord files and metadata saved to: /home/restorator/trader_test/2_results/tfrecord_universal_60_10_v1_20_features


In [ ]:
!python -m _tools.diagnose_prepared_data --model_name "{MODEL_NAME}"

--- 🩺 Диагностика подготовленных данных для модели: universal_60_10_v1_20_features ---
✅ Загружаю данные из: /home/restorator/trader_test/2_results/final_for_model/universal_60_10_v1_20_features/train_final.csv
Найдено 20 признаков. Провожу анализ...

🎉 Проверка завершена. Явных проблем (NaN, Inf, выбросы, константы) не найдено.
Статистика по всем признакам:
                              mean           std          min            max
ICS_26                 2225.177601  12008.377385     0.000762  231000.000000
imoex_close            2626.131329    679.106987  1237.430000    4287.520000
usdrub_close             70.068306     13.136364    30.758500     120.378500
sp500_close            3443.710325   1045.380266  1541.609985    5667.200195
imoex_volatility_20d      0.012658      0.009542     0.003086       0.137133
spread_hl                 0.038821      0.043361     0.000000       1.288994
vix_close                18.230464      7.082688     9.140000      82.690002
usdrub_volatility_20d  

In [ ]:
MODEL_NAME = f"universal_60_10_v1_20_features"
DATA_SOURCE_MODEL = f"{MODEL_NAME}"


!python -m _tools.train_universal_lstm \
    --model_name "{MODEL_NAME}" \
    --data_model_name "{DATA_SOURCE_MODEL}" \
    --use_gpu \
    --batch_size 8192 \
    --lr 1e-3 \
    --l2_reg 1e-5 \
    --verbose 1 \
    --leaky_relu_negative_slope 0.0

I0000 00:00:1759480925.307189  920786 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1759480926.357932  920786 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Использую данные от модели: 'universal_60_10_v1_20_features'
Обученная модель будет сохранена как: 'universal_60_10_v1_20_features'
✅ Метаданные загружены.
✅ Датасеты TFRecord готовы.
Текущий глобальный рекорд val_accuracy: 0.00000
--- Запуск 1 независимых сеансов обучения ---

========================= Начинаю итерацию №1/1 =========================
Создаю новую архитектуру модели LSTM...
E0000 00:00:1759480928.309561  920855 ptx_com

In [4]:
MODEL_NAME = f"universal_60_10_v1_35_features"

!python -m _tools.feature_selector_lstm \
  --model_name "{MODEL_NAME}" \
  --data_dir "2_results/final_for_model/{MODEL_NAME}" \
  --strategy forward --max_features 64 \
  --lookback 60 --val_fraction 0.2 \
  --epochs 25 --batch_size 512 --lr 1e-2 --patience 6 --alpha 0.4

I0000 00:00:1759424244.291837  449080 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1759424245.462448  449080 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/restorator/trader_test/_tools/feature_selector_lstm.py", line 400, in <module>
    main(args)
  File "/home/restorator/trader_test/_tools/feature_selector_lstm.py", line 256, in main
    train_df = _read_csv_required(train_path)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/

In [7]:
UNIVERSE_FILE="universe.csv"
MODEL_NAME_PREFIX = "universal_60_10_v1"
MODEL_NAME = f"universal_60_10_v1_110_features_accuracy"
DATA_SOURCE_MODEL = f"{MODEL_NAME}"

!python -m _tools.feature_selector \
    --model_name "{MODEL_NAME}" \
    --top_n 160 \
    --corr_threshold 1.0 \
    --repeats 3 \
    --n_splits 6 \
    --use_gpu \
    --ranking_metric accuracy

Загрузка файлов с признаками из: /home/restorator/trader_test/2_results/features_processed
Заполнение пропусков: 100%|█████████████████| 150/150 [00:00<00:00, 1653.84it/s]

--- 🚀 Запуск отбора признаков по метрике: 'ACCURACY' ---

--- Повтор [1/3] ---
  Фолд [1/6]...
  Фолд [2/6]...
  Фолд [3/6]...
  Фолд [4/6]...
  Фолд [5/6]...
  Фолд [6/6]...

--- Повтор [2/3] ---
  Фолд [1/6]...
  Фолд [2/6]...
  Фолд [3/6]...
  Фолд [4/6]...
  Фолд [5/6]...
  Фолд [6/6]...

--- Повтор [3/3] ---
  Фолд [1/6]...
  Фолд [2/6]...
  Фолд [3/6]...
  Фолд [4/6]...
  Фолд [5/6]...
  Фолд [6/6]...

--- Рейтинг признаков по ВЛИЯНИЮ НА ТОЧНОСТЬ (положительные = полезные) ---
Порог корреляции >= 1.0, фильтрация пропускается.

--- Итоговый рейтинг (топ-20) ---
VWAP_D                   0.107342
ICS_26                   0.102417
BBL_5_2.0_2.0            0.022879
sp500_close              0.012105
usdrub_close             0.010192
imoex_close              0.007361
imoex_volatility_20d     0.006619
THERMOma_20_2_0.

In [8]:
UNIVERSE_FILE="universe.csv"
MODEL_NAME_PREFIX = "universal_60_10_v1"
MODEL_NAME = f"universal_60_10_v1_110_features_loss"
DATA_SOURCE_MODEL = f"{MODEL_NAME}"

!python -m _tools.feature_selector \
    --model_name "{MODEL_NAME}" \
    --top_n 160 \
    --corr_threshold 1.0 \
    --repeats 3 \
    --n_splits 5 \
    --use_gpu \
    --ranking_metric loss

Загрузка файлов с признаками из: /home/restorator/trader_test/2_results/features_processed
Заполнение пропусков: 100%|█████████████████| 150/150 [00:00<00:00, 1472.57it/s]

--- 🚀 Запуск отбора признаков по метрике: 'LOSS' ---

--- Повтор [1/3] ---
  Фолд [1/5]...
  Фолд [2/5]...
  Фолд [3/5]...
  Фолд [4/5]...
  Фолд [5/5]...

--- Повтор [2/3] ---
  Фолд [1/5]...
  Фолд [2/5]...
  Фолд [3/5]...
  Фолд [4/5]...
  Фолд [5/5]...

--- Повтор [3/3] ---
  Фолд [1/5]...
  Фолд [2/5]...
  Фолд [3/5]...
  Фолд [4/5]...
  Фолд [5/5]...

--- Рейтинг признаков по ВЛИЯНИЮ НА LOSS (отрицательные = вредные) ---
   loss_impact < 0: Вредный признак (удаление улучшает/уменьшает loss)
   loss_impact > 0: Полезный признак (удаление ухудшает/увеличивает loss)
Порог корреляции >= 1.0, фильтрация пропускается.

--- Итоговый рейтинг (топ-20) ---
PSARr_0.02_0.2          0.000000
vix_high_fear_regime    0.000000
is_rate_high            0.000000
cbr_rate                0.000000
cbr_rate_close          0.000000
T

In [3]:
UNIVERSE_FILE="universe.csv"
MODEL_NAME_PREFIX = "universal_60_10_v1"

!python -m _tools.compare_model_sharpness \
    --model_prefix "{MODEL_NAME_PREFIX}" \
    --batch_size 16384 \
    --n_probes 20 \
    --noise_scale 1e-0

I0000 00:00:1759223699.578565   26848 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1759223701.332215   26848 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
--- Сравнительный детерминированный анализ резкости для моделей с префиксом: 'universal_60_10_v1' ---
Найдено 9 моделей для сравнения.
Используются данные, соответствующие каждой модели.
Анализ моделей:   0%|                                     | 0/9 [00:00<?, ?it/s]E0000 00:00:1759223703.640016   26899 ptx_compiler_helpers.cc:132] *** WARNING *** Invoking ptxas with version 12.5.82, which corresponds to a CUDA version <=12.6.2. CUDA 